In [1]:
# import datasets.face_dataset
# import tensorflow as tf
# import tensorflow_datasets as tfds
# from batch import BatchDatasetForOD
from datasets.augmented_face_dataset.augmented_face_dataset import NUM_TRAIN_EXAMPLES
NUM_TRAIN_EXAMPLES

2022-11-14 14:25:41.445741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 14:25:42.092758: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


1304

In [159]:
import albumentations as A
import cv2
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import datasets.face_dataset
import numpy as np

dataset = tfds.load("face_dataset")
train_dataset = dataset["train"]

In [160]:
transform = A.Compose([
    A.HorizontalFlip(always_apply=True),
    A.RandomBrightnessContrast(always_apply=True, contrast_limit=0.2, brightness_limit=0.2),
    A.Rotate(always_apply=True, limit=70, rotate_method="ellipse")
],  bbox_params=A.BboxParams(format='pascal_voc', label_fields=["class_labels"]))

In [161]:
image = cv2.imread("/dl/ssd/datasets/face_data/train/face_image_0.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
bbox = [[691,1047,1490,1845]]
labels = [0]

In [162]:
xml_template ='''
<annotation>
	<folder>data_face</folder>
	<filename>{image_filename}</filename>
	<path>data_face\{image_filename}</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>{width}</width>
		<height>{height}</height>
		<depth>{channels}</depth>
	</size>
	<segmented>0</segmented>
	{object_xml}
</annotation>'''

In [163]:
object_template = '''
<object>
    <name>face</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <bndbox>
        <xmin>{x1}</xmin>
        <ymin>{y1}</ymin>
        <xmax>{x2}</xmax>
        <ymax>{y2}</ymax>
    </bndbox>
</object>'''

In [164]:
print(''.join([object_template, object_template]))


<object>
    <name>face</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <bndbox>
        <xmin>{x1}</xmin>
        <ymin>{y1}</ymin>
        <xmax>{x2}</xmax>
        <ymax>{y2}</ymax>
    </bndbox>
</object>
<object>
    <name>face</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <bndbox>
        <xmin>{x1}</xmin>
        <ymin>{y1}</ymin>
        <xmax>{x2}</xmax>
        <ymax>{y2}</ymax>
    </bndbox>
</object>


In [165]:
folder_path = '/dl/ssd/augmented_data'
for example in train_dataset:
    image_filename = 'augmented_' + example['image_filename'].numpy().decode('UTF-8')
    image_name = os.path.splitext(image_filename)[0]
    image = example["image"].numpy()
    height,width,channels = tf.shape(image).numpy()
    bboxes = example["objects"][:,1:] * tf.convert_to_tensor([[width, height, width, height]], dtype=tf.float32).numpy()
    labels = example["objects"][:,0].numpy().astype(np.int32)
    print(image_filename)
    transformed = transform(image=image, bboxes=bboxes, class_labels=labels)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    transformed_class_labels = transformed['class_labels']

    object_xmls = []
    for label, box in zip(transformed_class_labels, transformed_bboxes):
        x1,y1,x2,y2 = round(box[0]),round(box[1]),round(box[2]),round(box[3])
        object_xmls.append(object_template.format(x1=x1,y1=y1,x2=x2,y2=y2))
    all_object_xml = ''.join(object_xmls)
    xml_str = xml_template.format(image_filename=image_filename,
                                width=width, height=height,channels=channels,
                                object_xml=all_object_xml)
    tf.keras.utils.save_img(os.path.join(folder_path,image_filename), transformed_image,"channels_last")
    with open(os.path.join(folder_path,image_name+'.xml'),'w') as f:
        f.write(xml_str)

augmented_face_image_561.jpg
augmented_face_image_1016.jpg
augmented_face_image_847.jpg
augmented_face_image_690.jpg
augmented_face_image_188.jpg
augmented_face_image_352.jpg
augmented_face_image_147.jpg
augmented_face_image_300.jpg
augmented_face_image_1082.jpg
augmented_face_image_303.jpg
augmented_face_image_521.jpg
augmented_face_image_650.jpg
augmented_face_image_215.jpg
augmented_face_image_1042.jpg
augmented_face_image_1073.jpg
augmented_face_image_232.jpg
augmented_face_image_99.jpg
augmented_face_image_953.jpg
augmented_face_image_391.jpg
augmented_face_image_160.jpg
augmented_face_image_371.jpg
augmented_face_image_768.jpg
augmented_face_image_1058.jpg
augmented_face_image_697.jpg
augmented_face_image_223.jpg
augmented_face_image_738.jpg
augmented_face_image_150.jpg
augmented_face_image_922.jpg
augmented_face_image_154.jpg
augmented_face_image_964.jpg
augmented_face_image_790.jpg
augmented_face_image_426.jpg
augmented_face_image_251.jpg
augmented_face_image_419.jpg
augmented_

In [166]:
# New augmented dataset
import datasets.augmented_face_dataset

dataset = tfds.load("augmented_face_dataset")

2022-11-14 14:18:53.481786: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/augmented_face_dataset/1.0.0.incompleteXOF7IA/augmented_face_dataset-train…

Generating valid examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/augmented_face_dataset/1.0.0.incompleteXOF7IA/augmented_face_dataset-valid…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/augmented_face_dataset/1.0.0.incompleteXOF7IA/augmented_face_dataset-test.…

Dataset augmented_face_dataset downloaded and prepared to /root/tensorflow_datasets/augmented_face_dataset/1.0.0. Subsequent calls will reuse this data.
